import seaborn as sns
# ^^^ pyforest auto-imports - don't write above this line

# Imports

In [2]:
import warnings
from config import *
from functions import *
import pickle
sns.set(style="whitegrid")
import pandas as pd


Bad key "text.kerning_factor" on line 4 in
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# Queries

## How Many Goalies are There?

In [25]:
# 215 goalies have played in the NHL 
num_goalies_query = f"""
    SELECT COUNT(DISTINCT(player_id))
    FROM game_goalie_stats
"""
num_goalies_query_result = run_query(num_goalies_query)

In [26]:
num_goalies_query_result

[(215,)]

### 10+ games goalies

In [14]:
num_goalies_10_query = f"""
    SELECT player_id, COUNT(*)
    FROM game_goalie_stats a
    GROUP BY player_id
        HAVING COUNT(game_id) > 10
"""
num_goalies_10_query_result = run_query(num_goalies_10_query)

In [15]:
len(num_goalies_10_query_result)

145

In [16]:
num_goalies_10_query_result

[(8468524, 231),
 (8455710, 213),
 (8471734, 562),
 (8460806, 111),
 (8460703, 197),
 (8469608, 455),
 (8468685, 617),
 (8471695, 521),
 (8460712, 136),
 (8470594, 576),
 (8470657, 480),
 (8474550, 467),
 (8471469, 593),
 (8466141, 468),
 (8470645, 526),
 (8467950, 474),
 (8471679, 531),
 (8469514, 174),
 (8470216, 73),
 (8475311, 195),
 (8473607, 251),
 (8475683, 491),
 (8462052, 57),
 (8470880, 395),
 (8470860, 397),
 (8467511, 26),
 (8473972, 296),
 (8471239, 400),
 (8468540, 102),
 (8469548, 133),
 (8474765, 135),
 (8474690, 52),
 (8460705, 140),
 (8474651, 509),
 (8473541, 326),
 (8462161, 41),
 (8475215, 275),
 (8473503, 346),
 (8460535, 105),
 (8466339, 103),
 (8460704, 81),
 (8468011, 441),
 (8473523, 153),
 (8474596, 289),
 (8476434, 262),
 (8473434, 188),
 (8471750, 412),
 (8476883, 241),
 (8477293, 156),
 (8473553, 50),
 (8476899, 209),
 (8474889, 348),
 (8473461, 366),
 (8473575, 423),
 (8471227, 497),
 (8470140, 450),
 (8477202, 61),
 (8475883, 365),
 (8471306, 244),
 (847

In [27]:
# 145 goalies have played more than one game
num_goalies_10_query2 = f"""
    SELECT COUNT(DISTINCT(player_id))
    FROM game_goalie_stats
    WHERE player_id IN (
        SELECT player_id
        FROM game_goalie_stats a
        GROUP BY player_id
            HAVING (COUNT(game_id) > 10)
    )
"""
num_goalies_10_query_result2 = run_query(num_goalies_10_query2)

In [28]:
num_goalies_10_query_result2

[(145,)]

## How Many Giveaways were made by Goalies

In [30]:
# I don't think there's data for passes...giveaways?
count_of_giveaways_by_goalies = f"""
    SELECT COUNT(*)
    FROM game_plays gp
    INNER JOIN game_plays_players gpp
        ON gp.play_id = gpp.play_id
    WHERE gpp.player_id IN (
        SELECT DISTINCT(player_id)
        FROM game_goalie_stats ggs)
    AND gp.event = 'Giveaway'
    ;
"""
count_of_giveaways_by_goalies_query_result = run_query(count_of_giveaways_by_goalies)

In [31]:
giveaways_by_goalies_query_result

[(2386,)]

## Which goalie had the most giveaways?

In [36]:
most_goalie_giveaways = f"""
    SELECT gpp.player_id, COUNT(*)
    FROM game_plays gp
    INNER JOIN game_plays_players gpp
        ON gp.play_id = gpp.play_id
    WHERE gpp.player_id IN (
        SELECT DISTINCT(player_id)
        FROM game_goalie_stats ggs)
    AND gp.event = 'Giveaway'
    ;
"""
most_goalie_giveaways_query_result = run_query(most_goalie_giveaways)

ProgrammingError: 1140 (42000): In aggregated query without GROUP BY, expression #1 of SELECT list contains nonaggregated column 'nhldatabase.gpp.player_id'; this is incompatible with sql_mode=only_full_group_by